In [ ]:
!pip install -q torch transformers transformers langchain sentence-transformers tqdm openpyxl openai pandas datasets
!pip install yandexgpt-python

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from tqdm.auto import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import json
from huggingface_hub import InferenceClient
from transformers import pipeline
import asyncio
import requests
import time
import datasets
import random
import re
import inspect
from yandex_gpt import YandexGPT, YandexGPTConfigManagerForAPIKey


pd.set_option("display.max_colwidth", None)

In [ ]:
with open('dataset.json', encoding='utf-8') as file:
    data = json.load(file)

contents = []

for item in data["data"]:
    content =  item["description"]
    metadata = item["url"]
    contents.append(content)

In [248]:
# Setup configuration (input fields may be empty if they are set in environment variables)
config = YandexGPTConfigManagerForAPIKey(model_type="yandexgpt", catalog_id="<CATALOG_ID>", api_key="<API-KEY>")

# Instantiate YandexGPT
yandex_gpt = YandexGPT(config_manager=config)

# Async function to get completion
async def get_response_yandex_gpt_async( text: str):
    role="user"
    messages = [{"role": role, "text": text}]
    completion = await yandex_gpt.get_async_completion(messages=messages, timeout=200)
    return completion

def get_response_sync(prompt):
  return("это пример синхронной функции")

def answer_with_rag(prompt):
  url = "<RAG_ENDPOINT>"
  payload = json.dumps({
    "query": prompt
  })
  headers = {
    'Content-Type': 'application/json'
  }
  response = requests.request("POST", url, headers=headers, data=payload)
  text = json.loads(response.text)['text']
  links = json.loads(response.text)['links']
  return {
      "text": text,
      "links": links
  }

# **Генерация эталонных ответов из датасета**

In [ ]:
#Шаблон ответа будет форматироваться
QA_generation_prompt = """
Ваша задача - написать конкретный вопрос по базе знаний компании Тинькофф и ответ на него, учитывая контекст.
На ваш вопрос должен быть дан конкретный, краткий ответ в виде фактической информации из контекста.
Ваш конкретный вопрос должен быть сформулирован в том же стиле, что и вопросы, которые пользователи могут задать в поисковой системе.
Это означает, что в фактоидном вопросе НЕ ДОЛЖНО упоминаться что-то вроде «согласно отрывку» или «контекст». Вопросы и ответы должны быть на русском языке.
Оформите свой ответ в формате JSON следующим образом:
{{
"question": "ваш вопрос (Поле должно называть именно question)"
"answer": "ваш ответ на вопрос (Поле должно называться именно answer)"
}}
Ни в коем случае не добавляйте каких либо други данных в ответ. Строго придерживайтесь структуры.
Вот контекст
Контекст: {context}
Ответ:::
"""

In [ ]:
#n_generations - количество пар вопрос ответ
#contents - содержимое датасета
#llm_method - метод для получения ответа от модели, должен принимать string prompt и возвращать string ответ
#prompt_template - шаблон запроса к llm
async def generate_golds_qa(n_generations: int, contents, llm_method, prompt_template):
  N_GENERATIONS = n_generations

  print(f"Generating {N_GENERATIONS} QA couples...")

  outputs = []
  for sampled_context in tqdm(random.sample(contents, N_GENERATIONS)):
    if inspect.iscoroutinefunction(llm_method):
      output_QA_couple = await llm_method(
           prompt_template.format(context=sampled_context)
      )
    else:
      output_QA_couple = llm_method(
          prompt_template.format(context=sampled_context)
      )
    try:
        #question = output_QA_couple.split("**Question:**")[-1].split("**Answer:**")[0]
        #answer = output_QA_couple.split("**Answer:**")[-1]
        print(output_QA_couple)
        response = json.loads(output_QA_couple)

        answer = response['answer']
        question=response['question']
        assert len(answer) < 300, "Answer is too long"
        outputs.append(
            {
                "context": sampled_context,
                "question": question,
                "answer": answer
            }
        )
    except:
        print('error')
        continue

  return (outputs) #возвращает массив объектов контекст, вопрос, ответ



In [ ]:
outputs = await generate_golds_qa(150, contents, get_response_yandex_gpt_async, QA_generation_prompt)

Generating 150 QA couples...


  0%|          | 0/150 [00:00<?, ?it/s]

{
"question": "Как изменить пароль для КЭП?",
"answer": "Запомните или храните пароль в надежном месте: если вы меняли пароль и не можете вспомнить его, квалифицированную подпись придется выпускать заново."
}
{
"question": "Как найти совершенные и незавершенные платежи в Тинькофф Бизнесе?",
"answer": ""
}

*В мобильном приложении: Платежи → Черновики.*

*В личном кабинете: Счета и платежи → Главное → Черновики. Нажмите на нужный платеж, и вы увидите детали редактирования.*
error
{
"question": "Что такое сальдо ЕНС?",
"answer": "Сальдо ЕНС — это баланс на вашем едином налоговом счёте с учётом всех налогов, долгов, штрафов и переплат."
}
{
"question": "Как скачать отчет о чатах в Тинькофф?",
"answer": "Чтобы скачать отчет о чатах, нужно выполнить следующие шаги:
1. На вкладке «Поиск» в разделе «Коммуникации» → «Чаты»: найдите чаты по нужным вам параметрам.
2. В результатах поиска нажмите «Скачать отчет».
3. Выберите формат файла для скачивания — XLSX и CSV.
4. Отметьте галочкой те параме

In [ ]:
outputs

[{'context': 'В TQM перейдите на вкладку «Поиск» → «Коммуникации» → «Теги».',
  'question': 'Как перейти к тегам в TQM?',
  'answer': 'Перейдите на вкладку «Поиск» → «Коммуникации» → «Теги».'},
 {'context': 'В конструкторе сайтов можно массово управлять товарами: искать, сортировать, дублировать, переносить, скрывать и удалять товары, указывать их единицы измерения и категории. Как указать категорию товара Чтобы массово управлять товарами, перейдите в нужный сайт и нажмите «Товары» в конструкторе сайтов Тинькофф. Теперь вы можете: Введите название или артикул товара в поле «Поиск по названию или артикулу». Если хотите, чтобы в списке отображались товары одной категории, нажмите «Поиск по категориям» и выберите из списка нужную. Товары можно сортировать в алфавитном или обратном алфавитном порядке. Для этого нажмите «Наименование» в шапке списка товаров. Указать категорию и единицу измерения можно для одного или сразу нескольких товаров. Для этого поставьте галочки слева от нужных товар

In [ ]:
display(pd.DataFrame(outputs).head(1))

,context,question,answer
0,"Это пароль, который дополнительно защищает КЭП от использования другими людьми. Его устанавливают при выпуске электронной подписи на токене, и для всех одинаковых моделей одного производителя он стандартный. Пароль можно посмотреть в документах к токену или на сайте производителя. Мы рекомендуем сразу заменить стандартный пароль на свой: это значительно безопаснее. Как изменить пароль для КЭП Запомните или храните пароль в надежном месте: если вы меняли пароль и не можете вспомнить его, квалифицированную подпись придется выпускать заново. Как выпустить КЭП через Тинькофф",Как изменить пароль для КЭП?,"Запомните или храните пароль в надежном месте: если вы меняли пароль и не можете вспомнить его, квалифицированную подпись придется выпускать заново."


# Фильтруем пары вопрос-ответ через критикующую модель

In [ ]:
#Промпт оценки однозначности, темплейт, будет форматироваться

question_groundedness_critique_prompt = """
Вам будет дан контекст и вопрос.
Ваша задача - дать «общий рейтинг», оценивающий, насколько хорошо можно однозначно ответить на поставленный вопрос с учетом заданного контекста.
Оцените свой ответ по шкале от 1 до 5, где 1 означает, что на вопрос вообще нельзя ответить, учитывая контекст, а 5 - что на вопрос можно ответить четко и однозначно, учитывая контекст.

Изложите свой ответ следующим образом в формате json:

{{
  "evaluation": "ваше обоснование оценки, в виде текста"
  "total_rating": "ваша оценка в виде числа от 1 до 5"(число обязательно должно быть в кавычках!!!)
}}




Вы ДОЛЖНЫ указать значения для  'evaluation' и 'total_rating'  в своем ответе.
}}
Ни в коем случае не добавляйте каких либо други данных в ответ. Строго придерживайтесь структуры.
Теперь вопрос и контекст.

Вопрос: {question}\n
Контекст: {context}\n
Answer::: """
#Промпт оценки полезности, темплейт, будет форматироваться

question_relevance_critique_prompt = """
Вам будет дан вопрос.
Ваша задача - дать «общий рейтинг», отражающий, насколько полезным может быть этот вопрос для предпринимателей, использующих продукты банка «Тинькофф».
Дайте свой ответ по шкале от 1 до 5, где 1 означает, что вопрос не полезен вообще, а 5 - что вопрос чрезвычайно полезен.

Изложите свой ответ следующим образом в формате json:

{{
  "evaluation": "ваше обоснование оценки, в виде текста"
  "total_rating": "ваша оценка в виде числа от 1 до 5 число"(число обязательно должно быть в кавычках!!!)
}}




Вы ДОЛЖНЫ указать значения для  'evaluation' и 'total_rating'  в своем ответе.
}}
Ни в коем случае не добавляйте каких либо други данных в ответ. Строго придерживайтесь структуры.

Теперь вопрос.

Question: {question}\n
Answer::: """
#Промпт оценки зависимости от контекста, темплейт, будет форматироваться
question_standalone_critique_prompt = """
Вам будет предложен вопрос.
Ваша задача - дать «общий рейтинг», отражающий, насколько этот вопрос зависит от контекста.
Оцените свой ответ по шкале от 1 до 5, где 1 означает, что для понимания вопроса требуется дополнительная информация, а 5 - что вопрос имеет смысл сам по себе.
Например, если вопрос относится к определенной обстановке, например «в контексте» или «в документе», оценка должна быть 1.

Изложите свой ответ следующим образом в формате json:

{{
  "evaluation": "ваше обоснование оценки, в виде текста"
  "total_rating": "ваша оценка в виде числа от 1 до 5" (число обязательно должно быть в кавычках!!!)
}}




Вы ДОЛЖНЫ указать значения для  'evaluation' и 'total_rating' t в своем ответе.
}}
Ни в коем случае не добавляйте каких либо други данных в ответ. Строго придерживайтесь структуры.

Теперь вопрос.
Question: {question}\n
Answer::: """

In [ ]:
#outputs вопросы-ответы-контексты полученные в голдах выше
async def generate_critiques(outputs, llm_method):
  print("Generating critique for each QA couple...")
  for output in tqdm(outputs):

      if inspect.iscoroutinefunction(llm_method):
            evaluations = {
            "groundedness": await llm_method(
                question_groundedness_critique_prompt.format(
                    context=output["context"], question=output["question"]
                )
            ),
            "relevance": await llm_method(
                question_relevance_critique_prompt.format(question=output["question"]),
            ),
            "standalone": await  llm_method(
                question_standalone_critique_prompt.format(question=output["question"]),
            )
            }
      else:
            evaluations = {
            "groundedness": llm_method(
                question_groundedness_critique_prompt.format(
                    context=output["context"], question=output["question"]
                )
            ),
            "relevance": llm_method(
                question_relevance_critique_prompt.format(question=output["question"]),
            ),
            "standalone": llm_method(
                question_standalone_critique_prompt.format(question=output["question"]),
            )
      }
      try:
          for criterion, evaluation in evaluations.items():

              response = json.loads(evaluation)
              print(response)



              score, eval = (
                  int(response['total_rating']),
                   response['evaluation']
              )

              output.update(
                  {
                      f"{criterion}_score": score,
                      f"{criterion}_eval": eval,
                  }
              )
      except :
          print('error')
          continue
      generated_questions = pd.DataFrame.from_dict(outputs)
  return generated_questions

In [ ]:
result = await generate_critiques(outputs, get_response_yandex_gpt_async)

Generating critique for each QA couple...


  0%|          | 0/118 [00:00<?, ?it/s]

{'evaluation': 'В контексте описаны подробные инструкции по изменению пароля для КЭП, включая рекомендации по безопасному хранению нового пароля. Также упоминается о последствиях в случае забывания нового пароля.', 'total_rating': '4'}
{'evaluation': 'Вопрос о том, как изменить пароль для КЭП, является важным и полезным для предпринимателей, использующих продукты банка «Тинькофф».', 'total_rating': '4'}
{'evaluation': 'Вопрос понятен без дополнительного контекста, так как он содержит всю необходимую информацию для ответа.', 'total_rating': '5'}
{'evaluation': 'Сальдо ЕНС — это баланс на едином налоговом счёте, который учитывает все налоги, долги, штрафы и переплаты.', 'total_rating': '5'}
{'evaluation': 'Вопрос полезен для предпринимателей, которые хотят разобраться в основах налогообложения и понять, как работает их финансовое состояние.', 'total_rating': '4'}
{'evaluation': 'Вопрос имеет смысл сам по себе, так как определение сальдо ЕНС не требует дополнительных контекстных данных.',

In [221]:
result

,context,question,answer,groundedness_score,groundedness_eval,relevance_score,relevance_eval,standalone_score,standalone_eval
0,"Это пароль, который дополнительно защищает КЭП от использования другими людьми. Его устанавливают при выпуске электронной подписи на токене, и для всех одинаковых моделей одного производителя он стандартный. Пароль можно посмотреть в документах к токену или на сайте производителя. Мы рекомендуем сразу заменить стандартный пароль на свой: это значительно безопаснее. Как изменить пароль для КЭП Запомните или храните пароль в надежном месте: если вы меняли пароль и не можете вспомнить его, квалифицированную подпись придется выпускать заново. Как выпустить КЭП через Тинькофф",Как изменить пароль для КЭП?,"Запомните или храните пароль в надежном месте: если вы меняли пароль и не можете вспомнить его, квалифицированную подпись придется выпускать заново.",4,"В контексте описаны подробные инструкции по изменению пароля для КЭП, включая рекомендации по безопасному хранению нового пароля. Также упоминается о последствиях в случае забывания нового пароля.",4,"Вопрос о том, как изменить пароль для КЭП, является важным и полезным для предпринимателей, использующих продукты банка «Тинькофф».",5,"Вопрос понятен без дополнительного контекста, так как он содержит всю необходимую информацию для ответа."
1,"Сальдо ЕНС — баланс на вашем едином налоговом счете с учетом всех налогов, долгов, штрафов и переплат. Например, ИП нужно было перевести на транспортный налог 30 000 ₽ и уплатить два штрафа по 5000 ₽. Он перевел на ЕНС 40 000 ₽. Сальдо ЕНС будет: 40 000 ₽ − 30 000 ₽ − 10 000 ₽ = 0 ₽. Сальдо может быть положительным, отрицательным и нулевым. Положительное сальдо означает, что на ЕНС есть деньги. Такая ситуация возникает, если вы внесли на ЕНС больше денег, чем было нужно, или вы пополнили ЕНС, а налог еще не успели списать. Деньгами на ЕНС можно управлять: вернуть; оставить на ЕНС в счет будущих налогов, которые входят в ЕНП; Какие платежи входят в ЕНП зачесть в счет будущих налогов, которые не входят в ЕНП; заплатить налоги другого человека, ИП или компании. Как ИП вернуть переплату или зачесть ее в счет будущих платежей Отрицательное сальдо означает, что у вас есть долги по налогам, штрафы или пени. На долги по налогам могут начисляться пени, а в некоторых случаях и штрафы. Как начисляются пени, если ИП вовремя не уплатил налоги Нулевое сальдо означает, что вы заплатили ровно ту сумму налогов и взносов, которой ожидало от вас государство на данный момент. После перехода на ЕНС налоговая провела сверку и определила сальдо для всех налогоплательщиков. Подробнее о сверке после перехода на ЕНС Сейчас сальдо формируется автоматически. Его можно посмотреть в личном кабинете налогоплательщика, чтобы быстро убедиться, что у вас нет долгов или переплат.",Что такое сальдо ЕНС?,"Сальдо ЕНС — это баланс на вашем едином налоговом счёте с учётом всех налогов, долгов, штрафов и переплат.",5,"Сальдо ЕНС — это баланс на едином налоговом счёте, который учитывает все налоги, долги, штрафы и переплаты.",4,"Вопрос полезен для предпринимателей, которые хотят разобраться в основах налогообложения и понять, как работает их финансовое состояние.",5,"Вопрос имеет смысл сам по себе, так как определение сальдо ЕНС не требует дополнительных контекстных данных."
2,"Есть два способа: через Тинькофф и налоговую. Через Тинькофф. Для этого нужно подать заявку в личном кабинете Тинькофф Бизнеса, а получить КЭП — в удостоверяющем центре «Основание» в своем городе или в точке выдачи Тинькофф в Москве. Подробно о том, как выпустить КЭП через Тинькофф Через налоговую. Для этого нужно подать заявку онлайн или офлайн и получить подпись в налоговой инспекции. Подробно о том, как выпустить КЭП через налоговую Независимо от того, где была оформлена КЭП, она имеет одинаковую юридическую силу.",Где можно получить КЭП?,"Получить КЭП можно в удостоверяющем центре «Основание» в своем городе или в точке выдачи Тинькофф в Москве, а также в налоговой инспекции.",4,"

In [232]:
import pandas as pd
from datasets import Dataset

def filter_eval_ds(unfiltered_ds: pd.DataFrame):
    unfiltered_ds['answer'] = unfiltered_ds['answer'].astype(str)  # Convert answer to string to avoid ArrowTypeError
    generated_questions = unfiltered_ds.loc[
        (unfiltered_ds["groundedness_score"] >= 4) &
        (unfiltered_ds["relevance_score"] >= 4) &
        (unfiltered_ds["standalone_score"] >= 4)
    ]

    print(generated_questions)

    eval_dataset = Dataset.from_pandas(generated_questions, preserve_index=False)
    return eval_dataset

# Assuming result is the DataFrame you want to filter...
eval_dataset = filter_eval_ds(result)

In [233]:
eval_dataset = filter_eval_ds(result)

In [243]:
eval_dataset

Dataset({
    features: ['context', 'question', 'answer', 'groundedness_score', 'groundedness_eval', 'relevance_score', 'relevance_eval', 'standalone_score', 'standalone_eval'],
    num_rows: 54
})

# Прогоняем примеры запросов через наш RAG и подставляем к голдам, сохраняем в файл

In [249]:
def run_rag_tests(
    rag_method,
    eval_dataset: datasets.Dataset,
    output_file: str,
    verbose: Optional[bool] = True,
    test_settings: Optional[str] = None,  # To document the test settings used
):
    """Runs RAG tests on the given dataset and saves the results to the given output file."""
    try:  # load previous generations if they exist
        with open(output_file, "r") as f:
            print('open')
            outputs = json.load(f)
    except:
        print('outputs')
        outputs = []

    for example in tqdm(eval_dataset):
        print(example)
        question = example["question"]
        print(question)
        if question in [output["question"] for output in outputs]:
            continue

        answer = rag_method(question)["text"]
        print(answer)
        if verbose:
            print("=======================================================")
            print(f"Question: {question}")
            print(f"Answer: {answer}")
            print(f'True answer: {example["answer"]}')
        result = {
            "question": question,
            "true_answer": example["answer"],
            "generated_answer": answer,

        }
        if test_settings:
            result["test_settings"] = test_settings
        outputs.append(result)

        with open(output_file, "w", encoding='utf8') as f:
            json.dump(outputs, f, ensure_ascii=False)

In [251]:
run_rag_tests(answer_with_rag, eval_dataset, "output.json")

open


  0%|          | 0/54 [00:00<?, ?it/s]

{'context': 'Это пароль, который дополнительно защищает КЭП от использования другими людьми. Его устанавливают при выпуске электронной подписи на токене, и для всех одинаковых моделей одного производителя он стандартный. Пароль можно посмотреть в документах к токену или на сайте производителя. Мы рекомендуем сразу заменить стандартный пароль на свой: это значительно безопаснее. Как изменить пароль для КЭП Запомните или храните пароль в надежном месте: если вы меняли пароль и не можете вспомнить его, квалифицированную подпись придется выпускать заново. Как выпустить КЭП через Тинькофф', 'question': 'Как изменить пароль для КЭП?', 'answer': 'Запомните или храните пароль в надежном месте: если вы меняли пароль и не можете вспомнить его, квалифицированную подпись придется выпускать заново.', 'groundedness_score': 4, 'groundedness_eval': 'В контексте описаны подробные инструкции по изменению пароля для КЭП, включая рекомендации по безопасному хранению нового пароля. Также упоминается о посл

In [252]:
EVALUATION_PROMPT = """###Описание задания:
Дан впорос, ответ, который нужно оценить, эталонный ответ, который оценивается в 5 баллов, и рубрикатор, представляющий критерии оценки.
1. Напишите подробный отзыв, в котором оцените качество ответа строго по заданной рубрике, не оценивая в целом.
2. После написания отзыва поставьте оценку, которая представляет собой целое число от 1 до 5. При этом необходимо ссылаться на рубрику оценок.
3. Оформите свой ответ следующим образом:

Изложите свой ответ следующим образом в формате json:

{{
  "evaluation": "ваше подрбное обоснование оценки, в виде текста"
  "total_rating": "ваша оценка в виде числа от 1 до 5"(число обязательно должно быть в кавычках!!!)
}}




Вы ДОЛЖНЫ указать значения для  'evaluation' и 'total_rating'  в своем ответе.
}}
Ни в коем случае не добавляйте каких либо други данных в ответ. Строго придерживайтесь структуры.

4. Пожалуйста, не генерируйте никаких других открытий, закрытий и пояснений. Обязательно укажите [total_rating] в выводе.

###Вопрос для оценивания:
{instruction}

###Ответ для оценивания:
{response}

###Эталонный ответ (Score 5):
{reference_answer}

###Рубрики оценки:
[Является ли ответ правильным, точным и фактическим, основанным на эталонном ответе?]
Оценка 1: Ответ полностью неверен, бесполезен, неточен и/или не соответствует действительности.
Оценка 2: Ответ в основном неверный, бесполезен, неточный и/или не соответствует действительности.
Оценка 3: Ответ в некоторой степени правильный, точный и/или фактический.
Оценка 4: Ответ в основном правильный, точный и фактический.
Оценка 5: Ответ полностью правильный, точный и фактический.

###Отзыв:"""

In [253]:
import re
async def evaluate_answers(file_path: str, eval_prompt_template: str, evaluator_name, llm_method):
    with open(file_path, encoding='utf8') as f:
        outputs = json.loads(f.read())

    for output in outputs:
      print(output)
      eval_prompt = eval_prompt_template.format(
            instruction=output["question"],
            response=output["generated_answer"],
            reference_answer=output["true_answer"],
        )
      if inspect.iscoroutinefunction(llm_method):
        eval_result = await llm_method(eval_prompt)
      else:
        eval_result = llm_method(eval_prompt)
      print(eval_result)
      result = json.loads(eval_result)
      feedback = result['evaluation']
      score = result['total_rating']
      output[f"eval_score_{evaluator_name}"] = score
      output[f"eval_feedback_{evaluator_name}"] = feedback

      with open(file_path, "w") as f:
             json.dump(outputs, f, ensure_ascii=False)

# Run the evaluate_answers function using the existing event loop
file_path = "output.json"
prompt_template = EVALUATION_PROMPT
eval_name = "yagpt"

# Check if the event loop is already running
try:
    loop = asyncio.get_running_loop()
except RuntimeError:  # No event loop running
    loop = None

if loop and loop.is_running():
    # If there's already a running loop, we create a task in it
    task = loop.create_task(evaluate_answers(file_path, prompt_template, eval_name, get_response_yandex_gpt_async))
    # Optionally, we can wait for the task to complete
    await task
else:
    # If no loop is running, we can use asyncio.run
    asyncio.run(evaluate_answers(file_path, prompt_template, eval_name))

{'question': 'Как изменить пароль для КЭП?', 'true_answer': 'Запомните или храните пароль в надежном месте: если вы меняли пароль и не можете вспомнить его, квалифицированную подпись придется выпускать заново.', 'generated_answer': 'Запомните или храните пароль в надёжном месте: если вы меняли пароль и не можете вспомнить его, квалифицированную подпись придётся выпускать заново.'}
{
  "evaluation": "Ответ является точным и фактическим, так как он полностью повторяет эталонный ответ. Он не содержит дополнительной информации или ошибок.",
  "total_rating": "5"
}
{'question': 'Что такое сальдо ЕНС?', 'true_answer': 'Сальдо ЕНС — это баланс на вашем едином налоговом счёте с учётом всех налогов, долгов, штрафов и переплат.', 'generated_answer': 'Сальдо ЕНС — это баланс на вашем едином налоговом счёте с учётом всех налогов, долгов, штрафов и переплат. Оно может быть положительным, отрицательным или нулевым.\n\nПоложительное сальдо означает, что на ЕНС есть деньги. Это может произойти, если в

In [258]:
def calc_average_eval(file_path, eval_score_name):
  with open(file_path, encoding='utf8') as f:
        outputs = json.loads(f.read())
  eval_count = 0
  eval_sum = 0
  for eval in outputs:
      eval_count+=1
      eval_sum+=int(eval[eval_score_name])
  return eval_sum / eval_count

In [259]:
calc_average_eval('output.json', 'eval_score_yagpt')

4.333333333333333